In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
import math
sns.set()

In [2]:
stock_name='GOOGL'

In [3]:
df = pd.read_csv('../../dataset/5year/'+stock_name+'-5year.csv')
date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()
data=df.iloc[:,1:].values
print(data.shape)

(1259, 6)


In [4]:
#定义常量
rnn_unit=10       #hidden layer units
input_size=data.shape[1]    
output_size=1
lr=0.01         #学习率
tf.reset_default_graph()
#输入层、输出层权重、偏置
weights={
         'in':tf.Variable(tf.random_normal([input_size,rnn_unit])),
         'out':tf.Variable(tf.random_normal([rnn_unit,1]))
         }
biases={
        'in':tf.Variable(tf.constant(0.1,shape=[rnn_unit,])),
        'out':tf.Variable(tf.constant(0.1,shape=[1,]))
        }

In [ ]:
def get_data(batch_size=60,time_step=20,train_begin=0,train_end=487):
    batch_index=[]
        
    scaler_for_x=MinMaxScaler(feature_range=(0,1))  #按列做minmax缩放
    scaler_for_y=MinMaxScaler(feature_range=(0,1))
    scaled_x_data=scaler_for_x.fit_transform(data[:,:-1])
    scaled_y_data=scaler_for_y.fit_transform(data[:,-1])
    
    label_train = scaled_y_data[train_begin:train_end]
    label_test = scaled_y_data[train_end:]
    normalized_train_data = scaled_x_data[train_begin:train_end]
    normalized_test_data = scaled_x_data[train_end:]
    
    train_x,train_y=[],[]   #训练集x和y初定义
    for i in range(len(normalized_train_data)-time_step):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:4]
        y=label_train[i:i+time_step,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data)-time_step))
    
    size=(len(normalized_test_data)+time_step-1)//time_step  #有size个sample 
    test_x,test_y=[],[]  
    for i in range(size-1):
        x=normalized_test_data[i*time_step:(i+1)*time_step,:4]
        y=label_test[i*time_step:(i+1)*time_step]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i+1)*time_step:,:4]).tolist())
    test_y.extend((label_test[(i+1)*time_step:]).tolist())    
    
    return batch_index,train_x,train_y,test_x,test_y,scaler_for_y